In [1]:
import os
import sys 
os.chdir("/workspaces/dev/modules")
sys.path.append("/workspaces/dev/models/R-CNN")

In [ ]:
from tensorflow import keras
import tensorflow as tf

2025-03-03 15:54:58.384161: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-03 15:54:58.393751: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741017298.404720    2304 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741017298.407840    2304 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-03 15:54:58.419000: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
from Pills import DataLoader
from PillNet import PillNet

In [ ]:
BATCH_SIZE = 32
ANNOTATION_PATH = "/workspaces/dev/datasets/pills/data/annotations.json"
IMAGE_DIR = "/workspaces/dev/datasets/pills"
MODEL_PATH = "/workspaces/dev/models/R-CNN/checkpoints/pillnet.h5"

In [5]:
data_loader = DataLoader(BATCH_SIZE, IMAGE_DIR, ANNOTATION_PATH)

In [6]:
model = PillNet()
model.build(4, (128, 128, 3))
model.summary()

I0000 00:00:1741017461.326722    2304 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9502 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 Ti, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "pill_net"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ FeatureExtractor (Sequential)   │ (None, 32, 32, 16)     │        93,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ SegmentationHead (Sequential)   │ (None, 128, 128, 2)    │        27,794 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 120,898 (472.26 KB)

 Trainable params: 120,898 (472.26 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# 1. IoU (Intersection over Union)
def iou_metric(y_true, y_pred, smooth=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)  # Threshold 적용
    intersection = tf.reduce_sum(y_true * y_pred)  # 교집합
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection  # 합집합
    return (intersection + smooth) / (union + smooth)  # IoU 계산

# 2. Dice Coefficient (F1-score 기반)
def dice_metric(y_true, y_pred, smooth=1e-6):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2 * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

# 3. Pixel-wise Accuracy
def pixel_accuracy(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    correct = tf.reduce_sum(tf.cast(y_true == y_pred, tf.float32))  # 맞춘 픽셀 수
    total = tf.reduce_sum(tf.ones_like(y_true, dtype=tf.float32))  # 전체 픽셀 수
    return correct / total


In [ ]:
model.compile(['accuracy', iou_metric, dice_metric, pixel_accuracy])

In [8]:
checkpoint_cb = keras.callbacks.ModelCheckpoint(
  filepath="/workspaces/dev/models/R-CNN/checkpoints/pill_net.keras",
  monitor="val_loss",
  save_best_only=True,
  save_weights_only=False,
  verbose = 1
)

In [ ]:
early_stopping_cb = keras.callbacks.EarlyStopping(
  monitor="val_loss",
  patience=10,
  restore_best_weights=True
)

In [9]:
model.fit(
  data_loader.get_train_sequence(),
  validation_data=data_loader.get_valid_sequence(),
  epochs=100,
  callbacks=[checkpoint_cb, early_stopping_cb]
)

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


I0000 00:00:1741017466.675085    3001 service.cc:148] XLA service 0x7f5fdc018df0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1741017466.675149    3001 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
2025-03-03 15:57:46.728904: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1741017466.947581    3001 cuda_dnn.cc:529] Loaded cuDNN version 90300


    1/24110 ━━━━━━━━━━━━━━━━━━━━ 57:10:07 9s/step - loss: 0.6932

I0000 00:00:1741017473.245874    3001 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24110/24110 ━━━━━━━━━━━━━━━━━━━━ 0s 302ms/step - loss: 0.0067
Epoch 1: val_loss improved from inf to 0.00318, saving model to /workspaces/dev/models/R-CNN/checkpoints/pill_net.keras
24110/24110 ━━━━━━━━━━━━━━━━━━━━ 9247s 383ms/step - loss: 0.0067 - val_loss: 0.0032
Epoch 2/100
24110/24110 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - loss: 0.0029
Epoch 2: val_loss improved from 0.00318 to 0.00257, saving model to /workspaces/dev/models/R-CNN/checkpoints/pill_net.keras
24110/24110 ━━━━━━━━━━━━━━━━━━━━ 4731s 196ms/step - loss: 0.0029 - val_loss: 0.0026
Epoch 3/100
24110/24110 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - loss: 0.0025
Epoch 3: val_loss improved from 0.00257 to 0.00245, saving model to /workspaces/dev/models/R-CNN/checkpoints/pill_net.keras
24110/24110 ━━━━━━━━━━━━━━━━━━━━ 7220s 299ms/step - loss: 0.0025 - val_loss: 0.0025
Epoch 4/100
24110/24110 ━━━━━━━━━━━━━━━━━━━━ 0s 286ms/step - loss: 0.0023
Epoch 4: val_loss improved from 0.00245 to 0.00205, saving model to /workspaces/dev/models/R-CNN

KeyboardInterrupt: 